# Data Analytic Boot Camp - Project #1

## Too Many Fast Food Restaurants Around us?

### Facts about the "Obesity Epidemic" in the U.S.
### - Approximate 34% of adults and 15 - 20% of children in the U.S. are obese.
### - Obesity rates have continued to increase over the past 3 decades.
### - There are many factors contribute to the problem


<img src = "./Graphs/brfss_2017_obesity-overall.svg" />

### The objective of this project is to test for any correlation between population to fast food ratio and obesity or diabetes rate at the county level.

### This project is a replication of the research paper 
### "Examining the relationship between the food environment and adult diabetes prevalence by county economic and racial composition:  an ecological study"
### by Haynes-Maslow & Leone, Aug 09, 2017

# This study is based on the data collected in the year 2013
# Data Resouces:
### - Centers for Disease Control and Prevention (CDC)
### - U.S. Census Bureau
### - Simple Maps Geographic Data Products
### - Yelp (API search)

#### The merged data set ("final_data.csv") is saved in output folder

In [2]:
# Data Analytic Boot Camp 
# Project #1

# Dependencies
import pandas as pd
import random
import matplotlib.pyplot as plt
import os
import csv
import requests
import json
import numpy as np
import plotly.plotly as py
import plotly.tools as tls
from scipy import stats
from config_1 import ykey
from config_2 import username
from config_2 import plotkey

# Since the final data frame is alreayd saved, please use the following csvpath to 
# retrieve the data frame to avoid running the API search again.

csvpath = os.path.join(".", "Outputs", "final_data.csv")
data_df = pd.read_csv(csvpath)

# data_df.head()

In [2]:
# Convert data downloaded from CDC into Pandas data frame

csvpath_1 = os.path.join(".", "Resources", "DM_PREV_ALL_STATES.csv")
csvpath_2 = os.path.join(".", "Resources", "OB_PREV_ALL_STATES.csv")
csvpath_3 = os.path.join(".", "Resources", "co-est2017-alldata.csv")
csvpath_4 = os.path.join(".", "Resources", "uscitiesv1.4.csv")

diabetes_df = pd.read_csv(csvpath_1)
obesity_df = pd.read_csv(csvpath_2)
population_df = pd.read_csv(csvpath_3, encoding="latin-1")
location_df = pd.read_csv(csvpath_4)

# obesity_df.head()

C:\Users\lokma\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning:

Columns (4,6,7,8,9,10,12,15) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
# Extract the 2013 data for analysis
diabetes_13_df = diabetes_df[["Diagnosed Diabetes Prevalence", "Unnamed: 1", "Unnamed: 2",
                          "2013", "Unnamed: 67", "Unnamed: 70"]]


# Replace header with the first row
new_header = diabetes_13_df.iloc[0]
diabetes_13_df = diabetes_13_df[1:]
diabetes_13_df.columns = new_header


# diabetes_13_df.head()

In [4]:
# Extract the 2013 data for analysis
obesity_13_df = obesity_df[["Obesity Prevalence", "Unnamed: 1", "Unnamed: 2",
                          "2013", "Unnamed: 67", "Unnamed: 70"]]


# Replace header with the first row
new_headers = obesity_13_df.iloc[0]
obesity_13_df = obesity_13_df[1:]
obesity_13_df.columns = new_headers


# obesity_13_df.head()

In [5]:
# Take out "county" from each county
obesity_13_df["County"] = obesity_13_df["County"].str[:-7]
diabetes_13_df["County"] = diabetes_13_df["County"].str[:-7]

# diabetes_13_df.head()

# Step 1: Data Merging
## Merge the data sets downloaded from CDC, U.S. Census, and Simple Maps
#### Note: Data cannot be merged by the county names becasue there are duplicated county names in different States
#### Note: Data will be merged by using the unique county FIPS codes

In [6]:
# Merge the obesity and diabetes table with fips code

merge_table = pd.merge(diabetes_13_df, obesity_13_df, on="FIPS Codes")

merge_table = merge_table.drop(columns=["State_x", "County_x", "State_y", "County_y"])

# merge_table.head()

In [7]:
# Extract geo location information (lat & lng) for each county
# location_df.head()

name = []
state = []
fips = []
lat = []
lng = []
zips = []

for i in range(len(location_df["county_name"])):
    
    try:
        if location_df["county_name"][i] != location_df["county_name"][i+1]:
            name.append(location_df["county_name"][i])
            state.append(location_df["state_name"][i])
            fips.append(location_df["county_fips"][i])
            lat.append(location_df["lat"][i])
            lng.append(location_df["lng"][i])
            zips.append(location_df["zips"][i])
    except KeyError:
        print("continue")
    
    
location = {"County_Name": name, "State_Name": state,
           "FIPS Codes": fips, "lat": lat, "lng": lng, "Zip_Code": zips}

loca_df = pd.DataFrame(location)

# loca_df.head()
# print(len(loca_df))

continue


In [8]:
# Merge the previous merged table with location table
# The merged table is not merging with the location table properly
# Solution: Change the FIPS codes to integer and try to merge again.

# Change FIPS codes to integer for the merged table
merge_table["FIPS Codes"] = merge_table["FIPS Codes"].astype(int)
# merge_table.dtypes

# Change FIPS codes to integer for the merged table
loca_df["FIPS Codes"] = loca_df["FIPS Codes"].astype(int)
# loca_df.dtypes

# Merge the two tables by FIPS codes (inner merge)
merge_table2 = pd.merge(merge_table, loca_df, on="FIPS Codes")

# merge_table2.head()
# print(len(merge_table2))

In [9]:
# Merge the previous merged table with population table
# The population data separted the FIPS codes into "state" and "county" codes
# Need to create a new column with for the FIPS code

# Create a new columne for FIPS code
population_df["FIPS Codes"] = (population_df["STATE"] * 1000) + population_df["COUNTY"]

# Extract the population information for 2013 and FIPS code for merging
pop_df = population_df[["FIPS Codes", "POPESTIMATE2013"]]

# Merge the population data frame with the previous
merge_table3 = pd.merge(merge_table2, pop_df, on="FIPS Codes")

# merge_table3.head()

In [10]:
# Rename the column headers

data_df = merge_table3.rename(index=str, columns={
    "number_x": "Survey Diabetes",
    "percent_x": "Percent Diabetes",
    "age-adjusted percent_x": "Age-Adjusted Percent Diabetes",
    "number_y": "Survey Obesity",
    "percent_y": "Percent Obesity",
    "age-adjusted percent_y": "Age-Adjusted Percent Obesity",
    "POPESTIMATE2013": "Population"
})

# data_df.head()

# Step 2: Collecting the total number of fast food restaurants in each county via Yelp API
#### The API search is by the counties geographic location (latitude and longtitude)
#### The search will return the total number of fast food restaurants in that geographic location

In [ ]:
# Testing Yelp API resonse

# Yelp API key is stored in ykey
# headers = {"Authorization": "bearer %s" % ykey}
# endpoint = "https://api.yelp.com/v3/businesses/search"
# data_df["fast_food"] = ""

# # Define the parameters
# params = {"term": "fast food", "latitude": data_df["lat"][15],
#               "longitude": data_df["lng"][15], "radius": 30000, 
#               "categories": "fast food", "limit": 50}

# print(params)

# # Make a request to the Yelp API
# response = requests.get(url = endpoint, params = params, headers = headers)
# data_response = response.json()

# # Add the total counts of fast food stores to "total"
# data_df["fast_food"][15] = data_response["total"]
    
# data.head(20)

In [4]:
# Extract fast food restaurants data by county location from Yelp

# Yelp API key is stored in ykey
headers = {"Authorization": "bearer %s" % ykey}
endpoint = "https://api.yelp.com/v3/businesses/search"
data_df["fast_food"] = ""
total = []

for i in range(len(data_df)):
    print(data_df["lat"][i], data_df["lng"][i])

# Define the parameters
    params = {"term": "fast food", "latitude": data_df["lat"][i],
              "longitude": data_df["lng"][i], "radius": 30000, 
              "categories": "fast food", "limit": 50}
    print(params)

    try:
# Make a request to the Yelp API
        response = requests.get(url = endpoint, params = params, headers = headers)
        data_response = response.json()

# Add the total counts of fast food stores to "total"
        print(data_response["total"])
        total.append(data_response["total"])
        data_df["fast_food"][i] = data_response["total"]
        
    except KeyError:
        print("County not found!")
    

32.4321 -86.6582
{'term': 'fast food', 'latitude': 32.4321, 'longitude': -86.6582, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
72
30.3555 -87.5041
{'term': 'fast food', 'latitude': 30.3555, 'longitude': -87.5041, 'radius': 30000, 'categories': 'fast food', 'limit': 50}


C:\Users\lokma\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



297
31.7085 -85.6109
{'term': 'fast food', 'latitude': 31.7085, 'longitude': -85.6109, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
15
33.1468 -87.00299999999999
{'term': 'fast food', 'latitude': 33.1468, 'longitude': -87.00299999999999, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
209
33.9064 -86.5178
{'term': 'fast food', 'latitude': 33.9064, 'longitude': -86.5178, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
108
32.1395 -85.7141
{'term': 'fast food', 'latitude': 32.1395, 'longitude': -85.7141, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
7
31.5566 -86.8233
{'term': 'fast food', 'latitude': 31.5566, 'longitude': -86.8233, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
11
33.7602 -85.8117
{'term': 'fast food', 'latitude': 33.7602, 'longitude': -85.8117, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
149
33.009 -85.4721
{'term': 'fast food', 'latitude': 33.009, 'longitude': -85.4721, 'radius': 30000, 'categories': 'fas

12
33.3215 -86.1969
{'term': 'fast food', 'latitude': 33.3215, 'longitude': -86.1969, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
53
33.0567 -85.9312
{'term': 'fast food', 'latitude': 33.0567, 'longitude': -85.9312, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
18
33.1642 -87.3533
{'term': 'fast food', 'latitude': 33.1642, 'longitude': -87.3533, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
143
33.7323 -87.2802
{'term': 'fast food', 'latitude': 33.7323, 'longitude': -87.2802, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
35
31.1946 -88.0015
{'term': 'fast food', 'latitude': 31.1946, 'longitude': -88.0015, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
10
32.1861 -87.4699
{'term': 'fast food', 'latitude': 32.1861, 'longitude': -87.4699, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
7
34.1496 -87.4028
{'term': 'fast food', 'latitude': 34.1496, 'longitude': -87.4028, 'radius': 30000, 'categories': 'fast food', 'limit': 50}

1
36.4198 -93.5335
{'term': 'fast food', 'latitude': 36.4198, 'longitude': -93.5335, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
84
33.0657 -91.2134
{'term': 'fast food', 'latitude': 33.0657, 'longitude': -91.2134, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
6
33.9979 -93.1057
{'term': 'fast food', 'latitude': 33.9979, 'longitude': -93.1057, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
25
36.341 -90.2223
{'term': 'fast food', 'latitude': 36.341, 'longitude': -90.2223, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
11
35.5881 -91.9279
{'term': 'fast food', 'latitude': 35.5881, 'longitude': -91.9279, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
28
33.7565 -92.2401
{'term': 'fast food', 'latitude': 33.7565, 'longitude': -92.2401, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
6
33.0973 -93.1959
{'term': 'fast food', 'latitude': 33.0973, 'longitude': -93.1959, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
22
3

...

150
38.6404 -77.40899999999999
{'term': 'fast food', 'latitude': 38.6404, 'longitude': -77.40899999999999, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
1100
37.026 -80.5739
{'term': 'fast food', 'latitude': 37.026, 'longitude': -80.5739, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
111
38.7121 -78.1599
{'term': 'fast food', 'latitude': 38.7121, 'longitude': -78.1599, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
65
37.9603 -76.76100000000001
{'term': 'fast food', 'latitude': 37.9603, 'longitude': -76.76100000000001, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
13
37.3434 -79.9535
{'term': 'fast food', 'latitude': 37.3434, 'longitude': -79.9535, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
164
37.9062 -79.1973
{'term': 'fast food', 'latitude': 37.9062, 'longitude': -79.1973, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
106
38.4465 -78.9228
{'term': 'fast food', 'latitude': 38.4465, 'longitude': -78.9228, 'radius': 30

194
36.7335 -76.0435
{'term': 'fast food', 'latitude': 36.7335, 'longitude': -76.0435, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
690
38.0673 -78.9012
{'term': 'fast food', 'latitude': 38.0673, 'longitude': -78.9012, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
83
37.2691 -76.7075
{'term': 'fast food', 'latitude': 37.2691, 'longitude': -76.7075, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
323
39.1734 -78.1745
{'term': 'fast food', 'latitude': 39.1734, 'longitude': -78.1745, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
163
47.1259 -118.3821
{'term': 'fast food', 'latitude': 47.1259, 'longitude': -118.3821, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
9
46.1349 -117.1324
{'term': 'fast food', 'latitude': 46.1349, 'longitude': -117.1324, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
15
45.9346 -119.3503
{'term': 'fast food', 'latitude': 45.9346, 'longitude': -119.3503, 'radius': 30000, 'categories': 'fast food', 'li

51
40.6129 -80.5627
{'term': 'fast food', 'latitude': 40.6129, 'longitude': -80.5627, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
144
39.0418 -78.7447
{'term': 'fast food', 'latitude': 39.0418, 'longitude': -78.7447, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
24
39.3648 -80.307
{'term': 'fast food', 'latitude': 39.3648, 'longitude': -80.307, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
80
38.9059 -81.6607
{'term': 'fast food', 'latitude': 38.9059, 'longitude': -81.6607, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
22
39.4318 -77.8048
{'term': 'fast food', 'latitude': 39.4318, 'longitude': -77.8048, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
204
38.2962 -81.3037
{'term': 'fast food', 'latitude': 38.2962, 'longitude': -81.3037, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
89
39.11 -80.4085
{'term': 'fast food', 'latitude': 39.11, 'longitude': -80.4085, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
57


92
44.8202 -91.4948
{'term': 'fast food', 'latitude': 44.8202, 'longitude': -91.4948, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
80
45.9251 -88.2525
{'term': 'fast food', 'latitude': 45.9251, 'longitude': -88.2525, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
16
43.6525 -88.2604
{'term': 'fast food', 'latitude': 43.6525, 'longitude': -88.2604, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
81
45.7396 -88.7004
{'term': 'fast food', 'latitude': 45.7396, 'longitude': -88.7004, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
1
42.9011 -91.0972
{'term': 'fast food', 'latitude': 42.9011, 'longitude': -91.0972, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
7
42.7992 -89.7529
{'term': 'fast food', 'latitude': 42.7992, 'longitude': -89.7529, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
102
43.7097 -88.9907
{'term': 'fast food', 'latitude': 43.7097, 'longitude': -88.9907, 'radius': 30000, 'categories': 'fast food', 'limit': 50}


30
41.025 -107.4498
{'term': 'fast food', 'latitude': 41.025, 'longitude': -107.4498, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
0
42.4061 -105.3858
{'term': 'fast food', 'latitude': 42.4061, 'longitude': -105.3858, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
0
44.5892 -104.6966
{'term': 'fast food', 'latitude': 44.5892, 'longitude': -104.6966, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
1
43.238 -108.1068
{'term': 'fast food', 'latitude': 43.238, 'longitude': -108.1068, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
13
41.9627 -104.3863
{'term': 'fast food', 'latitude': 41.9627, 'longitude': -104.3863, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
8
43.7136 -108.1872
{'term': 'fast food', 'latitude': 43.7136, 'longitude': -108.1872, 'radius': 30000, 'categories': 'fast food', 'limit': 50}
2
43.7097 -106.6379
{'term': 'fast food', 'latitude': 43.7097, 'longitude': -106.6379, 'radius': 30000, 'categories': 'fast food', 'limit'

In [5]:
data_df.head()

,FIPS Codes,Survey Diabetes,Percent Diabetes,Age-Adjusted Percent Diabetes,Survey Obesity,Percent Obesity,Age-Adjusted Percent Obesity,County_Name,State_Name,lat,lng,Zip_Code,Population,fast_food,pop_ff_ratio
0,1001,5144,13.0,11.9,13513,34.1,33.8,Autauga,Alabama,32.4321,-86.6582,36003,54695,72,1334.024390
1,1003,15367,10.4,8.8,40365,27.4,27.2,Baldwin,Alabama,30.3555,-87.5041,36530,194736,297,1081.866667
2,1005,3813,18.4,16.9,9186,44.4,44.7,Barbour,Alabama,31.7085,-85.6109,36017,26947,15,2072.846154
3,1007,2557,14.8,13.8,6940,40.3,40.3,Bibb,Alabama,33.1468,-87.0030,35115,22503,209,83.966418
4,1009,5990,14.1,12.3,14768,34.6,34.9,Blount,Alabama,33.9064,-86.5178,35013 35121,57623,108,472.319672


# Step 3: Adding the Population to Fast Food Restaurant Ratio to the data
### To calculate the population to fastfood restaurnat ratio, we are using the following equation.

\begin{equation*}
Population FastFoodRatio = \frac{Total Population}{Total Fast Food Restaurant}
\end{equation*}

In [6]:
# Add fastfood_population_ratio to the data frame

# Change fast_food data type to integer for calculation
data_df["fast_food"] = data_df["fast_food"].astype(int)
# data.dtypes

# Add new colume for fastfood per population ratio
data_df["pop_ff_ratio"] = (data_df["Population"] / data_df["fast_food"])

# Calculate the mean of the ratio
print(sum(data_df["pop_ff_ratio"])/len(data_df["pop_ff_ratio"]))

inf


# Saving the final data set before cleaning
#### The reason to save the merged data set before cleaning is because we want make sure the data can be used for other purposes.
#### This final data set can be called out at the beginning of this jupyter notebook.
#### The total observation for the final data set has n = 3139

In [7]:
# Save the final data frame as csv file
data_df.to_csv("Outputs/final_data.csv", index=False, header=True)

# Note that the csv file final_data.csv contained all the works cover above this box.


# Step 4: Cleaning the data
#### Step 1: Dropping the counties that have 0 fast food restaurant
#### Step 2: Dropping the counties that has population fastfood ratio over 2000
#### Step 3: Check the summary statistics to see if any outliners in the data

In [2]:
# Start cleaning the outliners in the data

# Drop the location that has zero fast food restaurant
data_df = data_df[(data_df.fast_food != 0)]
# print(len(data_df))
# data_df.head()

# Result: data reduces to 2906 observations

# Drop the location that has population-fastfood ratio over 2000
data_df = data_df[(data_df.pop_ff_ratio <= 2000)]
# print(len(data_df))
# data_df.head()

# Result: data reduces to 2302 observations

# Categorize the counties by the population fastfood ratio
#### High Ratio Counties: pop_ff_ratio < 500
#### Mid Ratio Counties: 500 < pop_ff_ratio < 1500
#### Low Ratio Counties: 1500 < pop_ff_ratio < 2000

In [18]:
# Create ranking groups based on population fastfood ratio

# Create bins in which data will be held
bins_1 = [0, 500, 1500, 2000]

# Create the names for the three bins
group_names_1 = ["High_Ratio", "Mid_Ratio", "Low_Ratio"]

data_df["Rank"] = pd.cut(data_df["pop_ff_ratio"], bins_1, labels=group_names_1)

# data_df2.head(20)

In [19]:
# Define city type by precent of obesity

# Create bins in which data will be held
bins_2 = [0, 20, 40, 50]

# Create the names for the three bins
group_names_2 = ["Healthy_City", "Average_City", "Poor_Health_City"]

data_df["Obesity_Rank"] = pd.cut(data_df["Age-Adjusted Percent Obesity"], bins_2, labels=group_names_2)

# data_df.head(20)

In [20]:
# Define city type by precent of obesity

# Create bins in which data will be held
bins_3 = [0, 7.5, 15, 21]

# Create the names for the three bins
group_names_3 = ["Healthy_City", "Average_City", "Poor_Health_City"]

data_df["Diabetes_Rank"] = pd.cut(data_df["Age-Adjusted Percent Diabetes"], bins_3, labels=group_names_3)

# data_df.head(20)

In [21]:
# Separate the data frame into three city type groups
healthy_data = data_df.loc[data_df["Obesity_Rank"] == "Healthy_City"]
average_data = data_df.loc[data_df["Obesity_Rank"] == "Average_City"]
poor_data = data_df.loc[data_df["Obesity_Rank"] == "Poor_Health_City"]
# healthy_data.head()

In [ ]:
# Statistical Testing
#### T-test: Testing whether the high ratio counties has a higher obesity or debetes rate compare to an average county.
#### ANOVA: Testing whether the average obesity rate or diabetes rate are the same in the three ratio rank counties.
#### Regression Model: Checking for any correlation between population fastfood ratio to obestity or diabetes rate at county level.

In [ ]:
# Statistical Testing

# Separate the data frame into three ratio groups
high_ratio_data = data_df.loc[data_df["Rank"] == "High_Ratio"]
mid_ratio_data = data_df.loc[data_df["Rank"] == "Mid_Ratio"]
low_ratio_data = data_df.loc[data_df["Rank"] == "Low_Ratio"]
# high_ratio_data.head()


In [ ]:
# Statistical Testing

# Run t-test to check obesity rate in high ratio region is higher than the average county

(t_stat_obes, p_obes) = stats.ttest_ind(data_df["Age-Adjusted Percent Obesity"],
                                        high_ratio_data["Age-Adjusted Percent Obesity"])

print(t_stat_obes, p_obes)

In [ ]:
# Run t-test to check diabetes rate in high ratio region is higher than the average county

(t_stat_dia, p_dia) = stats.ttest_ind(data_df["Age-Adjusted Percent Diabetes"],
                                        high_ratio_data["Age-Adjusted Percent Diabetes"])

print(t_stat_dia, p_dia)

In [ ]:
# Put all t-test result into dataframe

t_test = {"Test":["Obesity", "Diabetes"],
               "T-Statistic": [t_stat_obes, t_stat_dia],
              "P-Value": [p_obes, p_dia]}

obes_t_results = pd.DataFrame(t_test)

obes_t_results

In [ ]:
# ANOVA Test

# Test to see if different popuation fastfood ratio make a different to obesity rate
(f_stat_obes, pv_obes) = stats.f_oneway(low_ratio_data["Age-Adjusted Percent Obesity"],
                                       mid_ratio_data["Age-Adjusted Percent Obesity"],
                                        high_ratio_data["Age-Adjusted Percent Obesity"])

print(f_stat_obes, pv_obes)

In [ ]:
# Test to see if different popuation fastfood ratio make a different to diabetes rate
(f_stat_dia, pv_dia) = stats.f_oneway(low_ratio_data["Age-Adjusted Percent Diabetes"],
                                       mid_ratio_data["Age-Adjusted Percent Diabetes"],
                                        high_ratio_data["Age-Adjusted Percent Diabetes"])

print(f_stat_dia, pv_dia)

In [ ]:
# Put all ANOVA results into dataframe

anova = {"Test":["Obesity", "Diabetes"],
               "F-Statistic": [f_stat_obes, f_stat_dia],
              "P-Value": [pv_obes, pv_dia]}

anova_results = pd.DataFrame(anova)

anova_results

In [ ]:
# Regression Modeling 
# Objective: 
# 1. test the correlation between population fastfood ratio and obesity rate or diabetes rate
# 2. If there is correlation, what is the direction of the correlation
# 3. How is the model describe the data (goodness of fit)

# Regression 1: 
# Dependent Variable: Obesity Rate
# Independent Variable: Population fastfood Ratio
coef_obs, inter_obs, r_sq_obs, pvalue_obs, std_err_obs = stats.linregress(data_df["pop_ff_ratio"], data_df["Age-Adjusted Percent Obesity"])

print("Coefficient = " + str(coef_obs))
print("Intercept = "+ str(inter_obs))
print("R-Square = " + str(r_sq_obs))
print("P-Value = " + str(pvalue_obs))
print("Standard Error = " + str(std_err_obs))

In [ ]:
# plot the graph
predicted_obs = coef_obs * data_df["pop_ff_ratio"] + inter_obs

In [ ]:
# Regression 2:
# Dependent Variable: Diabetes Rate
# Independent Variable: Population fastfood Ratio

coef_dia, inter_dia, r_sq_dia, pvalue_dia, std_err_dia = stats.linregress(data_df["pop_ff_ratio"], data_df["Age-Adjusted Percent Diabetes"])

print("Coefficient = " + str(coef_dia))
print("Intercept = "+ str(inter_dia))
print("R-Square = " + str(r_sq_dia))
print("P-Value = " + str(pvalue_dia))
print("Standard Error = " + str(std_err_dia))

In [ ]:
# plot the graph
predicted_dia = coef_dia * data_df["pop_ff_ratio"] + inter_dia

In [ ]:
# Put all Regression results into dataframe

reg = {"Results":["Coeffecient", "Intercept", "R^2", "P-Value", "Standard Error"],
               "Obesity": [coef_obs, inter_obs, r_sq_obs, pvalue_obs, std_err_obs],
              "Diabetes": [coef_dia, inter_dia, r_sq_dia, pvalue_dia, std_err_dia]}

reg_results = pd.DataFrame(reg)

reg_results

# Plotting the data
#### 1. plotting the frequency distribution for the key variables (obesity rate, diabetes rate, population fastfood ratio)
#### 2. plotting pop_ff_ratio against obesity rate and diabetes rate
#### 3. create pie chart to identify the proportion of the ratio rank in the data
#### 4. plotting the heat map in the U.S. by the population fastfood ratio for each county in the data
#### 5. plotting the regression line to show the fit of the regression model to the data variation

In [ ]:
# Ploting the frequency distribution 

# Plotting the population fast food ratio and obesity before cleaning
# plt.scatter(data_df["pop_ff_ratio"], data_df["Age-Adjusted Percent Obesity"])

# plt.title("Population FastFood Ratio vs Obesity Rate")
# plt.xlabel("Population FastFood Ratio (people per restaurant)")
# plt.ylabel("Obesity Rate by County (%)")
# plt.grid()
# plt.savefig("./Graphs/PFR_Obesity(Before).png")

In [ ]:
# Plotting the frequency distribution of Obesity Rate
plt.hist(data_df["Age-Adjusted Percent Obesity"], color="red")
plt.title("Frequency Distribution of Age-Adjusted Obesity Rate")
plt.xlabel("Age-Adjusted Percent Obesity (%)")
plt.ylabel("Frequency")
plt.grid()
plt.savefig("./Graphs/Obesity_Rate.png")

In [ ]:
# Plotting the frequency distribution of Diabetes Rate
plt.hist(data_df["Age-Adjusted Percent Diabetes"], color="green")
plt.title("Frequency Distribution of Age-Adjusted Diabetes Rate")
plt.xlabel("Age-Adjusted Percent Diabetes (%)")
plt.ylabel("Frequency")
plt.grid()
plt.savefig("./Graphs/Diabetes_Rate.png")

In [ ]:
# Plotting the population fast food ratio and obesity after cleaning
plt.scatter(data_df["pop_ff_ratio"], data_df["Age-Adjusted Percent Obesity"])

plt.title("Population FastFood Ratio vs Obesity Rate")
plt.xlabel("Population FastFood Ratio (people per restaurant)")
plt.ylabel("Obesity Rate by County (%)")
plt.grid()
plt.savefig("./Graphs/PFR_Obesity(After).png")

In [ ]:
# Plotting the population fast food ratio and diabetes after cleaning
plt.scatter(data_df["pop_ff_ratio"], data_df["Age-Adjusted Percent Diabetes"])

plt.title("Population FastFood Ratio vs Diabetes Rate")
plt.xlabel("Population FastFood Ratio (people per restaurant)")
plt.ylabel("Diabetes Rate by County (%)")
plt.grid()
plt.savefig("./Graphs/PFR_Diabetes(After).png")

In [ ]:
# Plotting the obesity rate and diabetes rate with population fastfood ratio
plt.scatter(data_df["Age-Adjusted Percent Obesity"], 
            data_df["Age-Adjusted Percent Diabetes"],
           s=(data_df["pop_ff_ratio"])/80)

plt.title("Obesity Rate vs. Diabetes Rate")
plt.xlabel("Age-Adjusted Percent Obesity (%)")
plt.ylabel("Age-Adjusted Percent Diabetes (%)")
plt.grid()
plt.savefig("./Graphs/Obes_Diab_Rate.png")

In [ ]:
# Plotting the counts of high, mid, and low density counties

region = ["High Ratio", "Mid Ratio", "Low Ratio"]
region_counts = [len(high_ratio_data), len(mid_ratio_data), len(low_ratio_data)]

x_axis = np.arange(len(region))

tick_location = [0, 1, 2]

plt.xticks(tick_location, region)

plt.xlim(-0.75, len(x_axis)-0.25)

plt.bar(x_axis, region_counts, color=["red", "orange", "green"],
       alpha=0.5, align="center")

for x, y in zip(x_axis, region_counts):
    plt.text(x, y, str(y))

plt.title("Number of County by Ratio Rank")
plt.xlabel("Population FastFood Ratio Rank")
plt.ylabel("Total Number of Counts")
plt.savefig("./Graphs/County_Rank_Count.png")

In [ ]:
# Create Pie Chart to show the proportion of the density level

# Pie Chart for Density Rank

pies = ["High Ratio", "Mid Ratio", "Low Ratio"]
pie_counts = [len(high_ratio_data), len(mid_ratio_data), len(low_ratio_data)]
colors = ["red", "orange", "green"]
explode = (0.2, 0.1, 0.1)

plt.pie(pie_counts, explode=explode, labels=pies, colors=colors,
       autopct="%1.2f%%", shadow=True, startangle=90)

plt.axis("equal")
plt.title("Proportion of Density Level Regions")
plt.savefig("./Graphs/County_Rank_Proportion.png")

In [ ]:
# Plotting regression line for obesity rate
plt.scatter(data_df["pop_ff_ratio"], data_df["Age-Adjusted Percent Obesity"])
plt.plot(data_df["pop_ff_ratio"], predicted_obs, color="red", label="Regression Line")

plt.title("Population FastFood Ratio vs Obesity Rate (Regression Line)")
plt.xlabel("Population FastFood Ratio (people per restaurant)")
plt.ylabel("Obesity Rate by County (%)")
plt.savefig("./Graphs/PFR_Obesity(Regression_Line).png")

In [ ]:
# Plotting regression line for diabetes rate
plt.scatter(data_df["pop_ff_ratio"], data_df["Age-Adjusted Percent Diabetes"])
plt.plot(data_df["pop_ff_ratio"], predicted_dia, color="orange", label="Regression Line")

plt.title("Population FastFood Ratio vs Diabetes Rate (Regression Line)")
plt.xlabel("Population FastFood Ratio (people per restaurant)")
plt.ylabel("Diabetes Rate by County (%)")
plt.savefig("./Graphs/PFR_Diabetes(Regression_Line).png")

In [ ]:
# Plotting the obesity rate and diabetes rate with population fastfood ratio by ranked counties
plt.scatter(high_ratio_data["Age-Adjusted Percent Obesity"], 
            high_ratio_data["Age-Adjusted Percent Diabetes"],
           s=(high_ratio_data["pop_ff_ratio"])/200, color="red")

plt.scatter(mid_ratio_data["Age-Adjusted Percent Obesity"], 
            mid_ratio_data["Age-Adjusted Percent Diabetes"],
           s=(mid_ratio_data["pop_ff_ratio"])/200, color="orange")

plt.scatter(low_ratio_data["Age-Adjusted Percent Obesity"], 
            low_ratio_data["Age-Adjusted Percent Diabetes"],
           s=(low_ratio_data["pop_ff_ratio"])/200, color="green")

plt.title("Obesity Rate vs. Diabetes Rate")
plt.xlabel("Age-Adjusted Percent Obesity (%)")
plt.ylabel("Age-Adjusted Percent Diabetes (%)")
plt.savefig("./Graphs/Obes_Diab_Rate_Ranked.png")

In [23]:
# Creating heatmap to shows the population fastfood ratio

tls.set_credentials_file(username="lokmannorman", api_key="PW9XooYRuaUJmkIcEJGO")

scl = [[0, "rgb(225,40,0)"], [500, "rgb(225,130,0)"], 
       [1500, "rgb(235,250,50)"], [2000, "rgb(80,220,155)"]]

data = [dict(type="scattergeo", locationmode="USA-states", lon=data_df["lng"] , lat=data_df["lat"] , 
             marker=dict(size=5, autocolorscale=False, colorscale=scl, color=data_df["pop_ff_ratio"] , 
                         colorbar=dict(title="Popluation & Fast Food Ratio")))]

layout = dict(title="Population & Fast Food Ratio Map", colorbar=True, geo=dict(scope="usa", projection=dict(type="alber usa"), 
                                               landcolor="rgb(180,200,200)", subunitcolor="rgb(50,50,50)",
                                               countrycolor="rgb(50,50,50)", countrywidth=0.5, subunitweight=0.5))

fig = dict(data=data, layout=layout)

py.iplot(fig,validate=False, filenamen="Fast Food Restaurant Heat Map")

C:\Users\lokma\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

